# Baseline - Spark with Hive
Here we load and write the parquet file to Minio in the traditional way, first by writing directly to Minio as parquet, then saving as a Hive table

## Importing Required Libraries
We will be importing `SparkSession` and `os`, which is used to read environment variable for the Minio access key and secret.

We also set some styling to display tables better.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import date_format, col
import os
from timeit import default_timer as timer

# this is to better display pyspark dataframes
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

## Setting up the Spark Session
Here we are creating the Spark session with configuration to connect to the hive metastore, and Minio instance. We also add in some conifuration to tune the s3a connection, based on the configuration defined [here](https://spot.io/blog/improve-apache-spark-performance-with-the-s3-magic-committer/) and [here](https://blog.min.io/migrating-from-hadoop-without-rip-and-replace/). 

In [6]:
spark = SparkSession.builder \
  .appName("hive") \
  .config("spark.driver.memory", "4g") \
  .config("spark.executor.memory", "4g") \
  .config("spark.sql.warehouse.dir", "s3a://warehouse/spark-hive/") \
  .config("spark.hadoop.hive.metastore.uris", "thrift://hive-metastore:9083") \
  .config("spark.jars", "/opt/extra-jars/hadoop-aws.jar,/opt/extra-jars/aws-sdk-bundle.jar,/opt/extra-jars/spark-hadoop-cloud.jar") \
  .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
  .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
  .config("spark.hadoop.fs.s3a.path.style.access", True) \
  .config("spark.hadoop.fs.s3a.access.key", os.getenv("AWS_ACCESS_KEY_ID")) \
  .config("spark.hadoop.fs.s3a.secret.key", os.getenv("AWS_SECRET_ACCESS_KEY")) \
  .config("spark.hadoop.fs.s3a.connection.maximum", 8192) \
  .config("spark.hadoop.fs.s3a.fast.upload.active.blocks", 2048) \
  .config("spark.hadoop.fs.s3a.fast.upload.buffer", "disk") \
  .config("spark.hadoop.fs.s3a.fast.upload", True) \
  .config("spark.hadoop.fs.s3a.max.total.tasks", 2048) \
  .config("spark.hadoop.fs.s3a.multipart.size", "512M") \
  .config("spark.hadoop.fs.s3a.multipart.threshold", "512M") \
  .config("spark.hadoop.fs.s3a.block.siz", "512M") \
  .config("spark.hadoop.fs.s3a.bucket.all.committer.magic.enabled", True) \
  .config("spark.hadoop.fs.s3a.committer.threads", 2048) \
  .enableHiveSupport() \
  .getOrCreate()

24/08/14 09:38:53 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


## Setting up the test parquet file as a dataframe

Here we read our test data into the Spark context

In [31]:
df = spark.read.parquet("file:///home/iceberg/workspace/downloaded-data/yellow_tripdata_2024-01.parquet")

Now we check the data to get an idea of the size, structure and the actual data.

In [32]:
print(f"Number of rows: {df.count()}")
print("Schema:")
df.printSchema()
print("Data:")
df.show(5)

Number of rows: 2964624
Schema:
root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)

Data:
+--------+--------------------+---------------------+---------------+-------------+----

Here we have a problem. In this dataset columns `tpep_pickup_datetime` and `tpep_dropoff_datetime` are of datatype `timestamp_ntz`. This datatype was added recently in Spark 3.4, and is not supported by Hive. This will then cause issues when trying to save it as a hive table in the Hive Metastore. As a quick solution, we can cast these columns to string, to keep thing simple.

In [1]:
df = (df.withColumn("tpep_pickup_datetime",  date_format(col("tpep_pickup_datetime"), "yyyy-MM-dd HH:mm:ss"))
       .withColumn("tpep_dropoff_datetime",  date_format(col("tpep_dropoff_datetime"), "yyyy-MM-dd HH:mm:ss")))

NameError: name 'df' is not defined

We then check the schema and data again, just to make sure.

In [34]:
df.printSchema()
df.show(5)

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+

## Writing to Minio directly
The first test is to write the data directy to Minio, in parquet format, and time it. This is the simplest way to get data into our datalake. The downside however is that, the only way to query it would be to know the exact location its saved to, to load it again into the Spark context, or to load it using pandas/Polars. To be able to query it like a table, would need some extra setup, as we will see later with Trino.

In [35]:
start = timer()
df.write.mode("overwrite").parquet('s3a://warehouse/spark/yellow_tripdata_2024-01.parquet')
end = timer()
print(end - start)

5.736305343998538


In my setup, it tool about 6 seconds. But your mileage may vary.

## Writing as a Hive table
Now we want to use the Hive metastore as the intermediary to save the data to Minio. The benefits of this is, this will also set it up as a queriable table using the Hive Metastore.

In [25]:
spark.sql("CREATE SCHEMA IF NOT EXISTS spark_hive LOCATION 's3a://warehouse/spark-hive/'")

DataFrame[]

In [36]:
start = timer()
df.write.mode('overwrite').format("parquet").saveAsTable("spark_hive.yellow_tripdata_2024_01")
end = timer()
print(end - start) 

6.760480457000085


## Reading with Trino

### Setting up the trino connection

In [8]:
from sqlalchemy import create_engine
import pandas as pd
from sqlalchemy.sql.expression import select, text

engine = create_engine('trino://user@trino:8080')
trino_conn = engine.connect()

### Reading Parquet data from Minio using Trino

First we have to setup the parquet location as an external table in our Hive Metastore. This will then allow Trino to use the Hive connection to run the queries.

In [14]:
# Create the schema/namesapce
trino_conn.execute(text("CREATE SCHEMA IF NOT EXISTS hive.spark WITH (LOCATION = 's3a://warehouse/spark/')"))

# create table
trino_conn.execute(text("""
    CREATE TABLE hive.spark.yellow_tripdata_2024_01 (
       vendorid integer,
       tpep_pickup_datetime varchar,
       tpep_dropoff_datetime varchar,
       passenger_count bigint,
       trip_distance double,
       ratecodeid bigint,
       store_and_fwd_flag varchar,
       pulocationid integer,
       dolocationid integer,
       payment_type bigint,
       fare_amount double,
       extra double,
       mta_tax double,
       tip_amount double,
       tolls_amount double,
       improvement_surcharge double,
       total_amount double,
       congestion_surcharge double,
       airport_fee double
    )
    WITH (
       external_location = 's3a://warehouse/spark/yellow_tripdata_2024-01.parquet',
       format = 'PARQUET'
    )
"""))


Then we can read it

In [16]:
start = timer()
spark_df_from_trino = pd.read_sql_query("select * from hive.spark.yellow_tripdata_2024_01", trino_conn)
end = timer()
print(end - start) 

65.41385233699839


In [17]:
spark_df_from_trino

,vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2024-01-24 15:17:12,2024-01-24 15:34:53,1.0,3.33,1.0,N,239,246,1,20.5,0.0,0.5,3.00,0.0,1.0,27.50,2.5,0.0
1,2,2024-01-24 15:52:24,2024-01-24 16:01:39,1.0,1.61,1.0,N,234,249,1,10.7,0.0,0.5,3.67,0.0,1.0,18.37,2.5,0.0
2,2,2024-01-24 15:08:55,2024-01-24 15:31:35,1.0,4.38,1.0,N,88,211,1,25.4,0.0,0.5,5.88,0.0,1.0,35.28,2.5,0.0
3,2,2024-01-24 15:42:55,2024-01-24 15:51:35,1.0,0.95,1.0,N,211,234,1,9.3,0.0,0.5,2.66,0.0,1.0,15.96,2.5,0.0
4,2,2024-01-24 15:52:23,2024-01-24 16:12:53,1.0,2.58,1.0,N,68,144,1,18.4,0.0,0.5,4.48,0.0,1.0,26.88,2.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2964619,2,2024-01-24 15:50:18,2024-01-24 16:03:51,1.0,1.71,1.0,N,114,234,1,13.5,0.0,0.5,1.75,0.0,1.0,19.25,2.5,0.0
2964620,2,2024-01-24 15:23:22,2024-01-24 15:33:41,1.0,1.51,1.0,N,90,79,1,11.4,0.0,0.5,1.00,0.0,1.0,16.40,2.5,0.0
2964621,2,2024-01-24 15:12:40,2024-01-24 15:18:46,1.0,0.96,1.0,N,48,142,1,7.9,0.0,0.5,1.19,0.0,1.0,13.09,2.5,0.0
2964622,2,2024-01-24 15:21:05,2024-01-24 15:39:01,1.0,2.41,1.0,N,142,263,2,17.0,0.0,0.5,0.00,0.0,1.0,21.00,2.5,0.0


### Reading Hive table

In [6]:
start = timer()
spark_hive_df_from_trino = pd.read_sql_query("select * from hive.spark_hive.yellow_tripdata_2024_01", trino_conn)
end = timer()
print(end - start) 

82.73632431299484


In [7]:
spark_hive_df_from_trino

,vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2024-01-24 15:17:12,2024-01-24 15:34:53,1.0,3.33,1.0,N,239,246,1,20.5,0.0,0.5,3.00,0.0,1.0,27.50,2.5,0.0
1,2,2024-01-24 15:52:24,2024-01-24 16:01:39,1.0,1.61,1.0,N,234,249,1,10.7,0.0,0.5,3.67,0.0,1.0,18.37,2.5,0.0
2,2,2024-01-24 15:08:55,2024-01-24 15:31:35,1.0,4.38,1.0,N,88,211,1,25.4,0.0,0.5,5.88,0.0,1.0,35.28,2.5,0.0
3,2,2024-01-24 15:42:55,2024-01-24 15:51:35,1.0,0.95,1.0,N,211,234,1,9.3,0.0,0.5,2.66,0.0,1.0,15.96,2.5,0.0
4,2,2024-01-24 15:52:23,2024-01-24 16:12:53,1.0,2.58,1.0,N,68,144,1,18.4,0.0,0.5,4.48,0.0,1.0,26.88,2.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2964619,2,2024-01-24 15:50:18,2024-01-24 16:03:51,1.0,1.71,1.0,N,114,234,1,13.5,0.0,0.5,1.75,0.0,1.0,19.25,2.5,0.0
2964620,2,2024-01-24 15:23:22,2024-01-24 15:33:41,1.0,1.51,1.0,N,90,79,1,11.4,0.0,0.5,1.00,0.0,1.0,16.40,2.5,0.0
2964621,2,2024-01-24 15:12:40,2024-01-24 15:18:46,1.0,0.96,1.0,N,48,142,1,7.9,0.0,0.5,1.19,0.0,1.0,13.09,2.5,0.0
2964622,2,2024-01-24 15:21:05,2024-01-24 15:39:01,1.0,2.41,1.0,N,142,263,2,17.0,0.0,0.5,0.00,0.0,1.0,21.00,2.5,0.0
